In [1]:

import cvzone
import cv2
import numpy as np
import math
import random
from cvzone.HandTrackingModule import HandDetector

# setup openCV capture and window size
capture = cv2.VideoCapture(0)
capture.set(3, 1280)
capture.set(4, 720)

detect = HandDetector(detectionCon=0.8, maxHands=1)


class snakeCVclass:
    def __init__(self, foodPath):
        self.point = []  # Points of the snake
        self.length = []  # Distance between points
        self.currentLength = 0  # Total snake length
        self.TotalAllowedLength = 150  # Total allowed length
        self.headPrevious = 0, 0  # Previous head point.
        self.smoothFactor = 5  # Adjust this for smoother movement
        self.prevPointIndex = (0, 0)  # Store the previous index finger position

        # Food initialization
        self.foodIMG = cv2.imread(foodPath, cv2.IMREAD_UNCHANGED)  # import the food image
        self.foodHeight, self.foodWidth, _ = self.foodIMG.shape
        self.foodLocation = 0, 0  # food location (or points)
        self.FoodLocationRandom()  # initialize the random function
        self.score = 0  # Game Score
        self.gameOver = False  # see if the game is over

    # set the food location
    def FoodLocationRandom(self):
        self.foodLocation = random.randint(100, 1000), random.randint(100, 600)

    def update(self, mainIMG, headCurrent):
        if self.gameOver:
            cvzone.putTextRect(mainIMG, "Game Over", [250, 350], scale=8, thickness=4, colorT=(255, 255, 255),
                               colorR=(0, 0, 255), offset=20)
            cvzone.putTextRect(mainIMG, f'Your Score: {self.score}', [250, 500], scale=8, thickness=5,
                               colorT=(255, 255, 255), colorR=(0, 0, 255), offset=20)
        else:
            # Apply smoothing by averaging previous position and current position
            currentX = (self.prevPointIndex[0] * (self.smoothFactor - 1) + headCurrent[0]) // self.smoothFactor
            currentY = (self.prevPointIndex[1] * (self.smoothFactor - 1) + headCurrent[1]) // self.smoothFactor
            self.prevPointIndex = (currentX, currentY)

            previousX, previousY = self.headPrevious
            self.point.append([currentX, currentY])
            distance = math.hypot(currentX - previousX, currentY - previousY)
            self.length.append(distance)
            self.currentLength += distance
            self.headPrevious = currentX, currentY

            # Reducing Length if current length exceeds the allowed length
            if self.currentLength > self.TotalAllowedLength:
                for i, length in enumerate(self.length):
                    self.currentLength -= length
                    self.length.pop(i)
                    self.point.pop(i)
                    if self.currentLength < self.TotalAllowedLength:
                        break

            # check if snake ate the food
            randomX, randomY = self.foodLocation
            if (randomX - self.foodWidth // 2 < currentX < randomX + self.foodWidth // 2 and
                    randomY - self.foodHeight // 2 < currentY < randomY + self.foodHeight // 2):
                self.FoodLocationRandom()
                self.TotalAllowedLength += 50
                self.score += 1

            # Draw the line
            if self.point:
                for i in range(1, len(self.point)):
                    cv2.line(mainIMG, tuple(self.point[i - 1]), tuple(self.point[i]), (0, 0, 255), 20)
                cv2.circle(mainIMG, tuple(self.point[-1]), 20, (200, 0, 200), cv2.FILLED)

            # collision check
            point = np.array(self.point[:-2], np.int32).reshape((-1, 1, 2))
            cv2.polylines(mainIMG, [point], False, (0, 200, 0), 3)
            minDist = cv2.pointPolygonTest(point, (currentX, currentY), True)

            if -1 <= minDist <= 1:
                self.gameOver = True
                self.point = []
                self.length = []
                self.currentLength = 0
                self.TotalAllowedLength = 150
                self.headPrevious = 0, 0
                self.prevPointIndex = (0, 0)
                self.FoodLocationRandom()

            # Draw food
            mainIMG = cvzone.overlayPNG(mainIMG, self.foodIMG,
                                        (randomX - self.foodWidth // 2, randomY - self.foodHeight // 2))

            # showScore
            cvzone.putTextRect(mainIMG, f'Your Score :{self.score}', [50, 80], scale=3, thickness=3, offset=10)

        return mainIMG


game = snakeCVclass("Donut.png")
restart_game = False

while True:
    success, img = capture.read()
    img = cv2.flip(img, 1)
    hand, img = detect.findHands(img, flipType=False)

    if hand:
        landmarkList = hand[0]['lmList']
        pointIndex = landmarkList[8][0:2]
        img = game.update(img, pointIndex)

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)
    if key == ord('r'):
        game.gameOver = False
        game.score = 0
        restart_game = True

    if restart_game:
        game = snakeCVclass("Donut.png")
        restart_game = False

    if key == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()
